In [ ]:
!pip install --pre flwr[simulation] torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision import transforms,datasets

DEVICE = torch.device("cpu")
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

Training on cpu


**Data loading**

load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set),

In [ ]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


**Model training/evaluation**

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(testloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

**Flower client**

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

**Server-side parameter initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-09-20 09:43:41,511 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flower 2022-09-20 09:43:48,120 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8039340443.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 4019670220.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8039340443.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 4019670220.0}
INFO flower 2022-09-20 09:43:48,134 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-09-20 09:43:48,146 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-09-20 09:43:48,152 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 20

(launch_and_fit pid=4650) [Client 8] fit, config: {}
(launch_and_fit pid=4649) [Client 9] fit, config: {}
(launch_and_fit pid=4650) Epoch 1: train loss 0.02867353893816471, accuracy 0.23866666666666667
(launch_and_fit pid=4649) Epoch 1: train loss 0.029494818300008774, accuracy 0.216
(launch_and_fit pid=4650) [Client 4] fit, config: {}


DEBUG flower 2022-09-20 09:44:07,041 | server.py:234 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:44:07,061 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-09-20 09:44:07,067 | server.py:170 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=4650) Epoch 1: train loss 0.028995981439948082, accuracy 0.236
(launch_and_evaluate pid=4650) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=4649) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=4650) [Client 5] evaluate, config: {}


DEBUG flower 2022-09-20 09:44:13,820 | server.py:184 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flower:evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:44:13,826 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-09-20 09:44:13,833 | server.py:220 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flower:fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_evaluate pid=4649) [Client 7] evaluate, config: {}
(launch_and_fit pid=4650) [Client 5] fit, config: {}
(launch_and_fit pid=4649) [Client 9] fit, config: {}
(launch_and_fit pid=4649) Epoch 1: train loss 0.024045240134000778, accuracy 0.3615555555555556
(launch_and_fit pid=4650) Epoch 1: train loss 0.02434549480676651, accuracy 0.358
(launch_and_fit pid=4649) [Client 3] fit, config: {}


DEBUG flower 2022-09-20 09:44:45,174 | server.py:234 | fit_round 3 received 3 results and 0 failures
DEBUG:flower:fit_round 3 received 3 results and 0 failures
DEBUG flower 2022-09-20 09:44:45,194 | server.py:170 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=4649) Epoch 1: train loss 0.02425691857933998, accuracy 0.366
(launch_and_evaluate pid=4649) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=4649) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=4650) [Client 6] evaluate, config: {}


DEBUG flower 2022-09-20 09:44:51,052 | server.py:184 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-09-20 09:44:51,059 | server.py:144 | FL finished in 62.88884788799987
INFO:flower:FL finished in 62.88884788799987
INFO flower 2022-09-20 09:44:51,063 | app.py:180 | app_fit: losses_distributed [(1, 0.06195797975858053), (2, 0.05519310816129049), (3, 0.051558896700541174)]
INFO:flower:app_fit: losses_distributed [(1, 0.06195797975858053), (2, 0.05519310816129049), (3, 0.051558896700541174)]
INFO flower 2022-09-20 09:44:51,072 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-09-20 09:44:51,076 | app.py:182 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-09-20 09:44:51,079 | app.py:183 | app_fit: metrics_centralized {}
INFO:flower:app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.06195797975858053
	round 2: 0.05519310816129049
	round 3: 0.051558896700541174

**Starting with a customized strategy**

We've seen the function start_simulation before. It accepts a number of arguments, amongst them the client_fn used to create FlowerClient instances, the number of clients to simulate num_clients, the number of rounds num_rounds, and the strategy.

In [ ]:
# Create FedAdam strategy
strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-09-20 09:44:51,141 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-09-20 09:44:54,768	WARNING services.py:2013 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 3913117696 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.12gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
INFO flower 2022-09-20 09:44:58,034 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 4021030502.0, 'memory': 8042061006.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.2'

(launch_and_fit pid=4892) [Client 6] fit, config: {}
(launch_and_fit pid=4893) [Client 3] fit, config: {}
(launch_and_fit pid=4893) Epoch 1: train loss 0.02879974991083145, accuracy 0.23933333333333334
(launch_and_fit pid=4892) Epoch 1: train loss 0.029161041602492332, accuracy 0.2228888888888889
(launch_and_fit pid=4893) [Client 1] fit, config: {}


DEBUG flower 2022-09-20 09:45:14,618 | server.py:234 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:45:14,640 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-09-20 09:45:14,651 | server.py:170 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=4893) Epoch 1: train loss 0.029315531253814697, accuracy 0.22866666666666666
(launch_and_evaluate pid=4893) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=4892) [Client 0] evaluate, config: {}


DEBUG flower 2022-09-20 09:45:21,372 | server.py:184 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flower:evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:45:21,378 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-09-20 09:45:21,384 | server.py:220 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flower:fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_fit pid=4893) Epoch 1: train loss 0.09455748647451401, accuracy 0.15333333333333332
(launch_and_evaluate pid=4893) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=4892) [Client 0] evaluate, config: {}


DEBUG flower 2022-09-20 09:46:06,536 | server.py:184 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures


(launch_and_evaluate pid=4893) [Client 8] evaluate, config: {}


INFO flower 2022-09-20 09:46:06,544 | server.py:144 | FL finished in 68.47128706500007
INFO:flower:FL finished in 68.47128706500007
INFO flower 2022-09-20 09:46:06,558 | app.py:180 | app_fit: losses_distributed [(1, 6.665569213867187), (2, 1.917873722076416), (3, 0.40250819587707526)]
INFO:flower:app_fit: losses_distributed [(1, 6.665569213867187), (2, 1.917873722076416), (3, 0.40250819587707526)]
INFO flower 2022-09-20 09:46:06,568 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-09-20 09:46:06,574 | app.py:182 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-09-20 09:46:06,580 | app.py:183 | app_fit: metrics_centralized {}
INFO:flower:app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 6.665569213867187
	round 2: 1.917873722076416
	round 3: 0.40250819587707526

**Server-side parameter evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-09-20 09:46:06,638 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flower 2022-09-20 09:46:13,947 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8041527707.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 4020763852.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8041527707.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 4020763852.0}
INFO flower 2022-09-20 09:46:13,966 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-09-20 09:46:13,978 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-09-20 09:46:13,981 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 20

Server-side evaluation loss 0.07366173887252808 / accuracy 0.098
(launch_and_fit pid=5142) [Client 4] fit, config: {}
(launch_and_fit pid=5143) [Client 9] fit, config: {}
(launch_and_fit pid=5143) Epoch 1: train loss 0.028732066974043846, accuracy 0.24066666666666667
(launch_and_fit pid=5142) Epoch 1: train loss 0.028697354719042778, accuracy 0.24733333333333332
(launch_and_fit pid=5143) [Client 2] fit, config: {}


DEBUG flower 2022-09-20 09:46:29,079 | server.py:234 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:46:29,103 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=5143) Epoch 1: train loss 0.02880057319998741, accuracy 0.23644444444444446


INFO flower 2022-09-20 09:46:29,329 | server.py:122 | fit progress: (1, 0.06107873702049255, {'accuracy': 0.284}, 14.613359012999354)
INFO:flower:fit progress: (1, 0.06107873702049255, {'accuracy': 0.284}, 14.613359012999354)
DEBUG flower 2022-09-20 09:46:29,338 | server.py:170 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

INFO flower 2022-09-20 09:47:12,168 | server.py:122 | fit progress: (3, 0.05251796293258667, {'accuracy': 0.372}, 57.45232515300086)
INFO:flower:fit progress: (3, 0.05251796293258667, {'accuracy': 0.372}, 57.45232515300086)
DEBUG flower 2022-09-20 09:47:12,178 | server.py:170 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05251796293258667 / accuracy 0.372
(launch_and_evaluate pid=5142) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=5143) [Client 2] evaluate, config: {}


DEBUG flower 2022-09-20 09:47:19,390 | server.py:184 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-09-20 09:47:19,394 | server.py:144 | FL finished in 64.67870370399942
INFO:flower:FL finished in 64.67870370399942
INFO flower 2022-09-20 09:47:19,402 | app.py:180 | app_fit: losses_distributed [(1, 0.060874710957209276), (2, 0.05504145614306132), (3, 0.05293793749809265)]
INFO:flower:app_fit: losses_distributed [(1, 0.060874710957209276), (2, 0.05504145614306132), (3, 0.05293793749809265)]
INFO flower 2022-09-20 09:47:19,410 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-09-20 09:47:19,415 | app.py:182 | app_fit: losses_centralized [(0, 0.07366173887252808), (1, 0.06107873702049255), (2, 0.05569546699523926), (3, 0.05251796293258667)]
INFO:flower:app_fit: losses_centralized [(0, 0.07366173887252808), (1, 0.06107873702049255), (2, 0.055695466

(launch_and_evaluate pid=5142) [Client 7] evaluate, config: {}


History (loss, distributed):
	round 1: 0.060874710957209276
	round 2: 0.05504145614306132
	round 3: 0.05293793749809265
History (loss, centralized):
	round 0: 0.07366173887252808
	round 1: 0.06107873702049255
	round 2: 0.05569546699523926
	round 3: 0.05251796293258667
History (metrics, centralized):
{'accuracy': [(0, 0.098), (1, 0.284), (2, 0.356), (3, 0.372)]}

**Sending/receiving arbitrary values to/from clients**

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [ ]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  # 
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-09-20 09:47:19,510 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flower 2022-09-20 09:47:26,579 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 4021132492.0, 'node:172.28.0.2': 1.0, 'memory': 8042264987.0, 'CPU': 2.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'object_store_memory': 4021132492.0, 'node:172.28.0.2': 1.0, 'memory': 8042264987.0, 'CPU': 2.0}
INFO flower 2022-09-20 09:47:26,589 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-09-20 09:47:26,603 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-09-20 09:47:26,618 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 20

Server-side evaluation loss 0.07379335117340088 / accuracy 0.094
(launch_and_fit pid=5387) [Client 0, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=5388) [Client 4, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=5388) Epoch 1: train loss 0.028915295377373695, accuracy 0.23555555555555555
(launch_and_fit pid=5387) Epoch 1: train loss 0.02883441187441349, accuracy 0.23666666666666666
(launch_and_fit pid=5388) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flower 2022-09-20 09:47:41,598 | server.py:234 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-09-20 09:47:41,621 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=5388) Epoch 1: train loss 0.029143869876861572, accuracy 0.222


INFO flower 2022-09-20 09:47:41,874 | server.py:122 | fit progress: (1, 0.06193368363380432, {'accuracy': 0.282}, 14.897764425999412)
INFO:flower:fit progress: (1, 0.06193368363380432, {'accuracy': 0.282}, 14.897764425999412)
DEBUG flower 2022-09-20 09:47:41,883 | server.py:170 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

INFO flower 2022-09-20 09:48:34,419 | server.py:122 | fit progress: (3, 0.049777178049087524, {'accuracy': 0.412}, 67.44306098699963)
INFO:flower:fit progress: (3, 0.049777178049087524, {'accuracy': 0.412}, 67.44306098699963)
DEBUG flower 2022-09-20 09:48:34,428 | server.py:170 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.049777178049087524 / accuracy 0.412
(launch_and_evaluate pid=5387) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=5388) [Client 9] evaluate, config: {}


DEBUG flower 2022-09-20 09:48:39,812 | server.py:184 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-09-20 09:48:39,817 | server.py:144 | FL finished in 72.84085263700035
INFO:flower:FL finished in 72.84085263700035
INFO flower 2022-09-20 09:48:39,826 | app.py:180 | app_fit: losses_distributed [(1, 0.061482506513595575), (2, 0.05197153854370117), (3, 0.04930508502324423)]
INFO:flower:app_fit: losses_distributed [(1, 0.061482506513595575), (2, 0.05197153854370117), (3, 0.04930508502324423)]
INFO flower 2022-09-20 09:48:39,835 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-09-20 09:48:39,838 | app.py:182 | app_fit: losses_centralized [(0, 0.07379335117340088), (1, 0.06193368363380432), (2, 0.0529113941192627), (3, 0.049777178049087524)]
INFO:flower:app_fit: losses_centralized [(0, 0.07379335117340088), (1, 0.06193368363380432), (2, 0.052911394

(launch_and_evaluate pid=5387) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 0.061482506513595575
	round 2: 0.05197153854370117
	round 3: 0.04930508502324423
History (loss, centralized):
	round 0: 0.07379335117340088
	round 1: 0.06193368363380432
	round 2: 0.0529113941192627
	round 3: 0.049777178049087524
History (metrics, centralized):
{'accuracy': [(0, 0.094), (1, 0.282), (2, 0.37), (3, 0.412)]}

**Scaling federated learning**

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust fraction_fit to 0.05, which means that only 5% of available clients (so 50 clients) will be selected for training each round:

In [ ]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-09-20 09:48:42,278 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-09-20 09:48:45,569	WARNING services.py:2013 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 3913117696 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.12gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
INFO flower 2022-09-20 09:48:49,059 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 4020821606.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 8041643214.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'object_store_memory': 402082

(launch_and_fit pid=5641) [Client 198, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5641) Epoch 1: train loss 0.00046276816283352673, accuracy 0.06666666666666667
(launch_and_fit pid=5641) Epoch 2: train loss 0.0004588626907207072, accuracy 0.17777777777777778
(launch_and_fit pid=5641) Epoch 3: train loss 0.0004556152271106839, accuracy 0.2222222222222222
(launch_and_fit pid=5641) [Client 261, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5641) Epoch 1: train loss 0.0004586466820910573, accuracy 0.2222222222222222
(launch_and_fit pid=5641) Epoch 2: train loss 0.000451223662821576, accuracy 0.24444444444444444
(launch_and_fit pid=5641) Epoch 3: train loss 0.00044412832357920706, accuracy 0.24444444444444444
(launch_and_fit pid=5641) [Client 876, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5641) Epoch 1: train loss 0.00045964011223986745, accuracy 0.17777777777777778
(launch_and_fi

(raylet) Spilled 3535 MiB, 36 objects, write throughput 186 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


(launch_and_fit pid=5641) [Client 182, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5641) Epoch 1: train loss 0.0004620575055014342, accuracy 0.13333333333333333
(launch_and_fit pid=5641) Epoch 2: train loss 0.0004585843125823885, accuracy 0.13333333333333333
(launch_and_fit pid=5641) Epoch 3: train loss 0.00045950827188789845, accuracy 0.13333333333333333
(launch_and_fit pid=5641) [Client 790, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5641) Epoch 1: train loss 0.0004588753799907863, accuracy 0.15555555555555556
(launch_and_fit pid=5641) 
(launch_and_fit pid=5641) Epoch 2: train loss 0.0004579337255563587, accuracy 0.15555555555555556
(launch_and_fit pid=5641) Epoch 3: train loss 0.00045398122165352106, accuracy 0.15555555555555556
(launch_and_fit pid=5642) [Client 730, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=5642) Epoch 1: train loss 0.0004627850139513612, accuracy 0.111

DEBUG flower 2022-09-20 09:49:58,805 | server.py:234 | fit_round 1 received 25 results and 0 failures


...

DEBUG flower 2022-09-20 09:56:17,519 | server.py:170 | evaluate_round 3: strategy sampled 50 clients (out of 1000)
DEBUG:flower:evaluate_round 3: strategy sampled 50 clients (out of 1000)


(launch_and_evaluate pid=5642) [Client 846] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 836] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 452] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 750] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 910] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 574] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 434] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 536] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 691] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 727] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 573] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 161] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 145] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 371] evaluate, config: {}
(launch_and_evaluate pid=5641) [Client 85] evaluate, config: {}
(launch_and_evaluate pid=5

DEBUG flower 2022-09-20 09:58:27,394 | server.py:184 | evaluate_round 3 received 50 results and 0 failures
DEBUG:flower:evaluate_round 3 received 50 results and 0 failures
INFO flower 2022-09-20 09:58:27,399 | server.py:144 | FL finished in 578.2782158930004
INFO:flower:FL finished in 578.2782158930004
INFO flower 2022-09-20 09:58:27,403 | app.py:180 | app_fit: losses_distributed [(1, 0.46103102016448977), (2, 0.46044590759277354), (3, 0.4565942201614379)]
INFO:flower:app_fit: losses_distributed [(1, 0.46103102016448977), (2, 0.46044590759277354), (3, 0.4565942201614379)]
INFO flower 2022-09-20 09:58:27,410 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-09-20 09:58:27,419 | app.py:182 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-09-20 09:58:27,422 | app.py:183 | app_fit: metrics_centralized {}
INFO:flower:app_fit: metrics_centralized {}


(launch_and_evaluate pid=5641) [Client 756] evaluate, config: {}
(launch_and_evaluate pid=5642) [Client 837] evaluate, config: {}


History (loss, distributed):
	round 1: 0.46103102016448977
	round 2: 0.46044590759277354
	round 3: 0.4565942201614379